This notebook demonstrates how chaining `TextModifier`s with `gridsearch.iter_strings` can help solve grid-based puzzles.

Here's a hypothetical puzzle (inspired by a puzzle in an old issue of Panda Magazine, which I solved by hand but then used as a test case when writing the grid searching functions):

------
# By Any Other Name
*You're always forgetting the terms for things.*
```
Y Q A Y Y C O Z F Y X U J H A  
C I T C R A D N A W I N K F T  
S Z R X I N X M R Y B S C R V  
L L W R Q S A X P O B P A E I  
L L J E D N N U F K R E T S K  
U W K G S G Z A Y Z E I T C Q  
O B T G L V H S B P A B L C M  
S D O A O S M O M U L L E I H  
M P M D V C Y P S L E E N D D  
K G O D Q P F E L Z G O S E F  
E G F N F V G K S O T R L N S  
Y X W N G A Y S J V J P L T W  
Y E S O E Q D N W F F R A E L  
I O W L O O S C T E H G J X A  
Z U A C L O B A U K U V M Y C
```
* Batavia (7)
* Black Panther Homeland (7)
* "Good luck!" (5 1 3)
* Imre Lipschitz (7)
* Indescribable (11)
* Nobel Prize-winner Shinya (8)
* Owl Parrots (7)
* Random Mishap (5 8)
* Slow Drip (7)
* Sudden Strike (5 6)
* Ty and Tandy (5 3 6)
------

So you poke at this for a bit, and you see answers for a few of these clues - you happen to know that Imre Lipschitz changed his last name to Lakatos, that the Indonesian capital of Batavia is now called Jakarta, and that Ty Johnson and Tandy Bowen are the alter-egos of Marvel superhero duo Cloak And Dagger.

You don't see any of these in the grid, but you do see "clonnddagger" running up the fourth column, so you guess (correctly) that every answer contains the trigram "aka", but that trigram has been replaced by another letter before putting them into the actual grid.

You could now find the answers for the other clues, hunt through the grid for them, and work from there. But maybe you're eager to move on to other puzzles, and you'd rather have Python do the grunt work of searching the grid for you!

So first, you parse the grid into a numpy array, using `puztool.parse_grid`, or the shorter alias `puztool.pg`:

In [1]:
from puztool import pg
grid = pg('''
Y Q A Y Y C O Z F Y X U J H A  
C I T C R A D N A W I N K F T  
S Z R X I N X M R Y B S C R V  
L L W R Q S A X P O B P A E I  
L L J E D N N U F K R E T S K  
U W K G S G Z A Y Z E I T C Q  
O B T G L V H S B P A B L C M  
S D O A O S M O M U L L E I H  
M P M D V C Y P S L E E N D D  
K G O D Q P F E L Z G O S E F  
E G F N F V G K S O T R L N S  
Y X W N G A Y S J V J P L T W  
Y E S O E Q D N W F F R A E L  
I O W L O O S C T E H G J X A  
Z U A C L O B A U K U V M Y C
'''.lower())

You know from the enumerations that the shortest answers are 7 letters long, which means the shortest strings you care about in this grid will be 5 characters long. So you can use `iter_strings` to find all 5+-character strings in the grid. The returned values will be `Result` objects where `.val` is the found string and `.provenance` is a tuple of `(start, end)` showing the coordinates of the first and last letters of that word in the grid. There are 5920 strings here, so let's just see a few of them:

In [2]:
from puztool.gridsearch import iter_strings
for i,s in enumerate(iter_strings(grid, len=(5,None))):
    if not i%1000: # only show every 1000th because there are a LOT
        print(s.val, s.provenance)

yqayy (FromGrid(start=(0, 0), end=(0, 4)),)
xnixr (FromGrid(start=(2, 6), end=(2, 2)),)
uwkgsgzayzeit (FromGrid(start=(5, 0), end=(5, 12)),)
oypfn (FromGrid(start=(7, 7), end=(11, 3)),)
egfnfvgksotrl (FromGrid(start=(10, 0), end=(10, 12)),)
fvozlupzkoywy (FromGrid(start=(12, 9), end=(0, 9)),)


Now we want to look at all strings that can be produced by taking a string from this and replacing a single character with `'aka'`. We can write this as a `puztool.TextModifier` - a function that takes a `Result` object and returns an iterable of `Result` objects, and automatically knows how to do things like chain with other modifiers. Here's that modifier:

In [3]:
def add_aka(result):
    s = result.val
    for i in range(len(s)):
        yield result.extend(s[:i]+'aka'+s[i+1:], (s, s[i]))

`Result.extend(val, provenance)` returns a new `Result` with the new value but with the new provenance *appended to* the old provenance. In this case, our `add_aka` processor is adding both the original string and which letter was replaced to the provenance chain so we can refer to it later. Thus, a single result looks like this:

In [4]:
result = (iter_strings(grid, len=(5,None)) | add_aka).first()
print(repr(result))

Result(val='akaqayy', provenance=(FromGrid(start=(0, 0), end=(0, 4)), ('yqayy', 'y')))


Finally, we can restrict the output to words or phrases in a wordlist using the `puztool.In` modifier as a filter. `puztool.lists.<name>` fetches a `WordList` object derived from `data/wordlists/<name>.txt`; this library doesn't ship with any word lists because they're all enormous, but I use a list stored as `npl.txt` that is just the NPL's `allwords.txt` with punctuation, spaces, etc. stripped out. So we can join `add_aka` to a filter with:

In [5]:
from puztool import lists
from puztool.pipeline import Pipeline
check = Pipeline.from_item_mod(add_aka) | lists.npl

Now we can run this on the full list of strings in the grid. Since the return values are `Result` objects with same-length provenances, it's helpful to unpack them into a pandas DataFrame so that they render nicely in this notebook:

In [8]:
p = iter_strings(grid, len=(5,None)) | add_aka | lists.npl
p.df()

AttributeError: 'Pipeline' object has no attribute 'df'

In [6]:
import pandas as pd
results = iter_strings(grid, len=(5, None) ) | check.all()
results = pd.DataFrame([r.val, *r.provenance] for r in results)
results

,0,1,2
0,yamanaka,"(0, 9)->(5, 4)","(yamans, s)"
1,unspeakable,"(0, 11)->(8, 11)","(unspeible, i)"
2,wakanda,"(1, 9)->(1, 5)","(wanda, a)"
3,freakaccident,"(1, 13)->(11, 13)","(fresccident, s)"
4,speakable,"(2, 11)->(8, 11)","(speible, i)"
5,breakaleg,"(3, 10)->(9, 10)","(brealeg, a)"
6,jakarta,"(4, 2)->(0, 2)","(jwrta, w)"
7,sneakattack,"(9, 12)->(1, 12)","(snelttack, l)"
8,kakapos,"(10, 7)->(6, 7)","(kepos, e)"
9,lakatos,"(10, 12)->(10, 8)","(lrtos, r)"


The above chart shows all the words or phrases found in the grid, followed by the coordinates where the word was located, then followed by the actual string that was in the grid and the letter that needs to be replaced by `'aka'` to yield the answer to the clue. Obviously there are a few extras - "speakable" and "leakage" aren't answers, they're just substrings of the answers "unspeakable" and "leakages" that happen to be words themselves.

But you can pick out the correct answers and their locations from this chart easily, or, if you think of it, you might try resorting these by the answers and looking at the extra letters:

In [7]:
results.sort_values(0)

,0,1,2
5,breakaleg,"(3, 10)->(9, 10)","(brealeg, a)"
12,cloakanddagger,"(14, 3)->(3, 3)","(clonnddagger, n)"
3,freakaccident,"(1, 13)->(11, 13)","(fresccident, s)"
6,jakarta,"(4, 2)->(0, 2)","(jwrta, w)"
8,kakapos,"(10, 7)->(6, 7)","(kepos, e)"
9,lakatos,"(10, 12)->(10, 8)","(lrtos, r)"
10,leakage,"(13, 3)->(9, 7)","(leage, a)"
11,leakages,"(13, 3)->(8, 8)","(leages, a)"
7,sneakattack,"(9, 12)->(1, 12)","(snelttack, l)"
4,speakable,"(2, 11)->(8, 11)","(speible, i)"


Even with the extra words in the `DataFrame`, it's easy to read the text `answeralias` in those extra letters. The answer to the puzzle, therefore, is **alias**.